# Assignment 3: SMILE Experiment
## Computational Methods in Psychology and Neuroscience
### Psychology 4215/7215 --- Fall 2023

# Objectives

Upon completion of this assignment, the student will have:

1. Used the list generation code to make experimental blocks.

2. Created a full-fledged experiment for collecting data.


# Assignment

* Write SMILE code in a Jupyter notebook (after making a copy and renaming it to have your userid in the title --- e.g., A03_SMILE_Experiment_mst3k).

## Details

Your assignment is to turn the lists generated by code from the previous assignment into an experiment. As a reminder, regardless of whether you selected option 1 or option 2, this is a recognition memory experiment. This means that participants will study a list of items one at a time, and then, after a short delay, be tested for their memory of those items. In the test phase of each block, participants will see the study items again, along with an equal number of new items, and for each item they must specify whether the item is an old target item (i.e., one that was on the study list) or a new lure item. 

The high level structure of the experiment is as follows:

- Present the participant some instructions explaining the task
- Optionally provide some practice making responses
- Loop over the blocks of study--test lists

Each block of study--test lists will have the following structure:

- Wait for the participant to press a key to start the block
- Loop over the study list presenting the study items, one at a time
- Wait for a delay (we may eventually fill this with some simple math problems)
- Loop over the test list to present the test items, one at a time, waiting for a keyboard response on each item

Each study item trial will:

- Present the item for a specified duration (this should be a configuration variable at the top of your code)
- Wait an inter-stimulus duration plus some amount of jitter (these, too, should be config variables)
- Log the stimulus information, including when it appeared on the screen

Each test item trial will:

- Present the item on the screen (with either a Label or Image state) until the participant makes a keyboard response of either the key you have selected to indicate the item is "old" or the key that indicates the item is "new"
- Log the stimulus information, including when the stimulus appeared on the screen, when the participant made their response, and what response they made

It is possible to write the entire experiment in one big state machine, but it may be easier to break up these different sections into subroutines.

Be sure to refer to the class notebooks to help guide how to do all the steps above. We have some code below to help you get started.

  
* ***When you are done, save this notebook as HTML (`File -> Download as -> HTML`) and upload it to the matching assignment on UVACollab.***  

In [2]:
# Load in the most common SMILE states
import csv
import random
from copy import deepcopy

import A02_ListGen_script as LG  # List Gen
import numpy as np
from smile.common import *
from smile.math_distract import MathDistract
from smile.startup import InputSubject

# enter configuration variables here (including the listgen variables)
## List gen
# # My list gen params
# lg_block = 2
# lg_subs = 1
# lg_block_params = {
#     "locs": ["indoor", "outdoor"],
#     "condition_types": ["1p", "massed-rep", "spaced-rep"],
#     "rep_types": [2],
#     "distance_types": [np.arange(3, 7)],
#     "ntrials": 6,
#     "test_length": 12,
#     "old_prop": 0.5,
#     "lure_types": ["lure"],
#     "time_blocks": 1,  # not used :(
# }
# lg_filename_dict = {"indoor": "indoor.csv", "outdoor": "outdoor.csv"}

# Per's list gen params
lg_pool_files = {"indoor": "indoor.csv", "outdoor": "outdoor.csv"}
lg_rep_conds = ["once", "massed", "spaced"]
lg_loc_conds = ["indoor", "outdoor"]
lg_spaced_range = (4, 9)
lg_num_reps = 1
lg_num_blocks = 2
num_tries = 1000

## Experiment
INST_TEXT = """[u][size=40]SPACED REP INSTRUCTIONS[/size][/u]

In this task, you will see pictures and your job is to remember them for a test later. 
    
Press ENTER key to continue."""
INST_FONT_SIZE = 45
INST_STUDY = """[u][size=40]STUDY PHASE[/size][/u]

This is the study phase of the task. The images will advance automatically. 

Please focus on each image and try to commit it to memory
    
Press ENTER key to continue."""
INST_MATH = """[u][size=40]MATH SECTION[/size][/u]

Here let's do some math!

Press "F" if the equation is correct
Press "J" if the equation is incorrect
    
Press ENTER key to continue."""
INST_TEST = """[u][size=40]TEST PHASE[/size][/u]

This is the test phase of the task. You will see images and be asked if they are new or old. 

The old images are ones that you have seen on the last study session. 
The new images are ones that you have never seen before. 

Press "F" if the image is OLD
Press "J" if the image is NEW
    
Press ENTER key to continue."""

END_TEXT = """[u][size=40]THANK YOU[/size][/u]

Thanks for participating! 
    
Press ENTER key to close."""
RESP_KEYS = ["F", "J"]
RESP_MAP = {"target": "F", "lure": "J"}
STIM_PATH = "./stimuli/images/"
STIM_DUR = 1
STIM_JITTER = 0
STUDY_ISI = 0.25
STUDY_JITTER = 0.0
TEST_ISI = 0.5
TEST_JITTER = 0.0

# Distraction piece
NUM_VARS = 3
MIN_NUM = 1
MAX_NUM = 9
MAX_PROBS = 50
DURATION = 2
STUDY_TEST_WAIT = 1

[INFO   ] [Logger      ] Record log in C:\Users\student\.kivy\logs\kivy_23-10-05_52.txt
[INFO   ] [Kivy        ] v2.2.1
[INFO   ] [Kivy        ] Installed at "c:\Users\student\anaconda3\envs\smile\lib\site-packages\kivy\__init__.py"
[INFO   ] [Python      ] v3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:34:57) [MSC v.1936 64 bit (AMD64)]
[INFO   ] [Python      ] Interpreter at "c:\Users\student\anaconda3\envs\smile\python.exe"
[INFO   ] [Logger      ] Purge log fired. Processing...
[INFO   ] [Logger      ] Purge finished!
[INFO   ] [Factory     ] 190 symbols loaded


[INFO   ] [Image       ] Providers: img_tex, img_dds, img_sdl2, img_pil (img_ffpyplayer ignored)
[INFO   ] [Text        ] Provider: sdl2
[INFO   ] [Window      ] Provider: sdl2
[INFO   ] [GL          ] Using the "OpenGL" graphics system
[INFO   ] [GL          ] GLEW initialization succeeded
[INFO   ] [GL          ] Backend used <glew>
[INFO   ] [GL          ] OpenGL version <b'4.6.0 - Build 26.20.100.7986'>
[INFO   ] [GL          ] OpenGL vendor <b'Intel'>
[INFO   ] [GL          ] OpenGL renderer <b'Intel(R) HD Graphics 520'>
[INFO   ] [GL          ] OpenGL parsed version: 4, 6
[INFO   ] [GL          ] Shading version <b'4.60 - Build 26.20.100.7986'>
[INFO   ] [GL          ] Texture max size <16384>
[INFO   ] [GL          ] Texture max units <32>
[INFO   ] [Window      ] auto add sdl2 input provider
[INFO   ] [Window      ] virtual keyboard not allowed, single mode, not docked
[CRITICAL] [Camera      ] Unable to find any valuable Camera provider. Please enable debug logging (e.g. add -

In [3]:

# call the listgen code to create your blocks
# (you can copy it in here from the solution notebook)
##### My list gen ######
# final_dict = LG.create_experiment(
#     lg_block_params, nBlocks=lg_block, nSubjects=lg_subs, filename_dict=lg_filename_dict
# )
# blocks_dict = final_dict["subj_0"]
# block_list = list(blocks_dict.values())


# pers code
# read all the pools into a dictionary
# Code to read in the pools
def read_and_shuffle(pool_file):
    """Read in and shuffle a pool."""
    # create a dictionary reader
    dr = csv.DictReader(open(pool_file, "r"))

    # read in all the lines into a list of dicts
    pool = [l for l in dr]

    # shuffle it so that the we get new items each time
    random.shuffle(pool)

    # report out some pool info
    print(pool_file, len(pool))

    # return the shuffled pool
    return pool


pools = {loc: read_and_shuffle(lg_pool_files[loc]) for loc in lg_loc_conds}
# create the conds
# fully crossed with all combos of val and rep
conds = []
for loc in lg_loc_conds:
    for rep in lg_rep_conds:
        # I decided to call the repetition condition cond
        conds.append({"loc": loc, "cond": rep})


# make a function for generating a block
# with a study and test list
def make_block():
    """Generate a block, uses global variables"""
    # loop and create the repeated conditions
    block_conds = []
    for i in range(lg_num_reps):
        # extend the trials with copies of the conditions
        block_conds.extend(deepcopy(conds))

    # try a number of times to satisfy the listgen
    # store temp items so that we can put them
    # back on the pools on failure
    temp_items = {k: [] for k in pools.keys()}
    for i in range(num_tries):
        print(i, end=": ")

        # put any temp items back into the pools
        for k in pools.keys():
            if len(temp_items[k]) > 0:
                pools[k].extend(temp_items[k])

        # shuffle the conds for that block
        random.shuffle(block_conds)

        # ensure there are enough non-spaced items at the end
        # loop backwards
        num_items = 0
        worked = False
        for c in block_conds[::-1]:
            num_items += 1
            if c["cond"] == "spaced":
                # make sure we have enough items
                if num_items >= lg_spaced_range[0]:
                    # it worked
                    worked = True

                # break and try again if needed
                break
        if not worked:
            print("x")
            continue

        # we've shuffled our conds, so fill them in with items
        # create the blank study list
        study_list = []
        for cond in block_conds:
            # add a place to fill
            study_list.append(None)
            if cond["cond"] in ["massed", "spaced"]:
                # append another
                study_list.append(None)

        test_list = []

        # loop over block conds and
        # add items to study/test lists
        worked = True  # let's be optimistic this time
        for cond in block_conds:
            # use the valence to grab study and test items
            study_item = pools[cond["loc"]].pop()
            test_item = pools[cond["loc"]].pop()

            # add those items to the temp_items
            temp_items[cond["loc"]].extend([study_item, test_item])

            # update with the cond info
            study_item.update(cond)
            test_item.update(cond)

            # add in relevant info for study and test
            study_item["pres_num"] = 1
            study_item["type"] = "target"
            test_item["type"] = "lure"
            test_item["pres_num"] = 1  # just so the keys match

            # insert the item into the study list
            if cond["cond"] == "once":
                # just insert in the first open spot
                try:
                    ind = study_list.index(None)
                except ValueError:
                    # no index found, so try again
                    worked = False
                    break

                # use the index to set the item
                study_item["lag"] = 0
                test_item["lag"] = 0
                study_list[ind] = study_item
                print("O", end="")

            elif cond["cond"] == "massed":
                # find the first index with two open spots
                success = False
                for ind in range(len(study_list) - 1):
                    if study_list[ind] is None and study_list[ind + 1] is None:
                        # add in the item
                        study_item["lag"] = 1
                        test_item["lag"] = 1
                        study_list[ind] = study_item
                        rep_item = deepcopy(study_item)
                        rep_item["pres_num"] = 2
                        study_list[ind + 1] = rep_item
                        success = True
                        print("M", end="")
                        break

                # test for failure
                if not success:
                    worked = False
                    break
            else:
                # cond is spaced
                # find the first index with open slots
                # for the second item
                success = False
                for ind in range(len(study_list) - lg_spaced_range[0]):
                    if study_list[ind] is None:
                        # see if we have an open space
                        pos_ind = []
                        for ind2 in range(
                            ind + lg_spaced_range[0], ind + lg_spaced_range[1]
                        ):
                            if ind2 < len(study_list) and study_list[ind2] is None:
                                pos_ind.append(ind2)
                        if len(pos_ind) > 0:
                            # pick from the options at random
                            ind2 = random.choice(pos_ind)
                            lag = ind2 - ind

                            # add in the item
                            study_item["lag"] = lag
                            test_item["lag"] = lag
                            study_list[ind] = study_item
                            rep_item = deepcopy(study_item)
                            rep_item["pres_num"] = 2
                            study_list[ind2] = rep_item
                            success = True
                            print("S", end="")
                            break

                # test for failure
                if not success:
                    worked = False
                    break

            # append them to the respective lists
            # study item is added to both study and test
            test_list.append(study_item)
            test_list.append(test_item)

        # if it worked, break
        if worked:
            print(" Success!")
            break
        else:
            print("X")

    if not worked:
        raise RuntimeError("Unable to generate list.")

    # must shuffle the test list
    random.shuffle(test_list)

    # make a dictionary to return
    block = {"study": study_list, "test": test_list}

    return block


# generate the proper number of blocks
block_list = []
for b in range(lg_num_blocks):
    block_list.append(make_block())

indoor.csv 335
outdoor.csv 309
0: x
1: x
2: x
3: x
4: x
5: SSMMOO Success!
0: x
1: x
2: SOSMMO Success!


In [4]:
# create an experiment instance
exp = Experiment(name="OLDNEW", show_splash=False, resolution=(1024, 768))


# # YOUR CODE HERE TO BUILD THE STATE MACHINE
# show the stimulus (will default to center of the screen)
@Subroutine
def studyTrial(self, block_num, trial_num, trial):
    # present stimulus
    stim = Image(
        source=STIM_PATH + trial["filename"],
        width=1400,
        height=1400,
        allow_stretch=True,
        keep_ratio=True,
    )
    # wait
    with UntilDone():
        Wait(STIM_DUR, STIM_JITTER)
    # trial ISI
    Wait(STUDY_ISI, STUDY_JITTER)

    Log(
        log_dict=trial,
        name="old-new-study",
        location=trial["loc"],
        condition=trial["cond"],
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        # TODO: stim offset time?
    )


@Subroutine
def testTrial(self, block_num, trial_num, trial):
    # present the stimulus
    stim = Image(
        source=STIM_PATH + trial["filename"],
        width=1400,
        height=1400,
        allow_stretch=True,
        keep_ratio=True,
    )

    with UntilDone():
        # make sure the stimulus has appeared on the screen
        Wait(until=stim.appear_time)

        # collect a response (with no timeout)
        kp = KeyPress(
            keys=RESP_KEYS,
            base_time=stim.appear_time["time"],
            correct_resp=Ref.object(RESP_MAP)[trial["type"]],
        )

    # wait the ISI with jitter
    Wait(Ref.object(TEST_ISI), Ref.object(TEST_JITTER))

    # # TODO: provide feedback to participant?
    # num_correct = Ref.object(num_correct) + kp.correct

    # log the result of the trial
    Log(
        name="old-new-test",
        log_dict=trial,
        block_num=block_num,
        trial_num=trial_num,
        stim_on=stim.appear_time,
        resp=kp.pressed,
        resp_time=kp.press_time,
        rt=kp.rt,
        correct=kp.correct,
    )


@Subroutine
def studyTestBlock(self, block_num, block_dict):
    # study block
    Label(
        text=INST_STUDY,
        font_size=INST_FONT_SIZE,
        text_size=(exp.screen.width * 0.75, None),
        markup=True,
    )
    with UntilDone():
        # Wait(3)
        KeyPress(keys=["ENTER"])
    with Loop(block_dict["study"]) as trial:
        studyTrial(block_num, trial.i, trial.current)

    # Interval block
    Wait(STUDY_TEST_WAIT)
    Label(
        text=INST_MATH,
        font_size=INST_FONT_SIZE,
        text_size=(exp.screen.width * 0.75, None),
        markup=True,
    )
    with UntilDone():
        # Wait(3)
        KeyPress(keys=["ENTER"])

    MathDistract(
        num_vars=NUM_VARS,
        min_num=MIN_NUM,
        max_num=MAX_NUM,
        max_probs=MAX_PROBS,
        duration=DURATION,
    )

    # test block
    Label(
        text=INST_TEST,
        font_size=INST_FONT_SIZE,
        text_size=(exp.screen.width * 0.75, None),
        markup=True,
    )
    with UntilDone():
        Wait(3)
        KeyPress(keys=["ENTER"])
    with Loop(block_dict["test"]) as trial:
        testTrial(block_num, trial.i, trial.current)

    # TODO: provide feedback on performance?


# InputSubject("OLDNEW")

### MAIN FLOW ###
Label(
    text=INST_TEXT,
    font_size=INST_FONT_SIZE,
    text_size=(exp.screen.width * 0.75, None),
    markup=True,
)
with UntilDone():
    # Wait(3)
    KeyPress(keys=["ENTER"])

with Loop(block_list) as block_dict:
    studyTestBlock(block_dict.i, block_dict.current)

Label(
    text=END_TEXT,
    font_size=INST_FONT_SIZE,
    text_size=(exp.screen.width * 0.75, None),
    markup=True,
)
with UntilDone():
    KeyPress(keys=["ENTER"])


# run the experiment
exp.run()

[INFO   ] [Base        ] Start application main loop
[INFO   ] [GL          ] NPOT texture support is available
[WARNING] Deprecated property "<BooleanProperty name=allow_stretch>" of object "<kivy.uix.image.Image object at 0x0000028D07A6CDD0>" has been set, it will be removed in a future version
[WARNING] Deprecated property "<BooleanProperty name=keep_ratio>" of object "<kivy.uix.image.Image object at 0x0000028D07A6CDD0>" was accessed, it will be removed in a future version
[INFO   ] [Base        ] Leaving application in progress...


In [7]:
from smile.log import log2dl
import pandas as pd

path = "data/OLDNEW/test000/20231005_235400/"

study = log2dl(path + "log_old-new-study_0.slog")
df_study = pd.DataFrame(study)
df_study

,location,condition,block_num,trial_num,stim_on_time,stim_on_error,log_time,filename,in_out,loc,cond,pres_num,type,lag,log_num
0,outdoor,spaced,0,0,6616.114862,0.0,6617.329494,out1376.jpg,outdoor,outdoor,spaced,1,target,6,0
1,indoor,spaced,0,1,6617.351497,0.0,6618.579494,in0110.jpg,indoor,indoor,spaced,1,target,8,0
2,outdoor,massed,0,2,6618.609611,0.0,6619.829494,out0075_new.jpg,outdoor,outdoor,massed,1,target,1,0
3,outdoor,massed,0,3,6619.868524,0.0,6621.079494,out0075_new.jpg,outdoor,outdoor,massed,2,target,1,0
4,indoor,massed,0,4,6621.110867,0.0,6622.329494,in0025.jpg,indoor,indoor,massed,1,target,1,0
5,indoor,massed,0,5,6622.353382,0.0,6623.579494,in0025.jpg,indoor,indoor,massed,2,target,1,0
6,outdoor,spaced,0,6,6623.619788,0.0,6624.829494,out1376.jpg,outdoor,outdoor,spaced,2,target,6,0
7,outdoor,once,0,7,6624.854118,0.0,6626.079494,out0146_new.jpg,outdoor,outdoor,once,1,target,0,0
8,indoor,once,0,8,6626.103606,0.0,6627.329494,in0400.jpg,indoor,indoor,once,1,target,0,0
9,indoor,spaced,0,9,6627.371049,0.0,6628.579494,in0110.jpg,indoor,indoor,spaced,2,target,8,0


In [8]:
test = log2dl(path + "log_old-new-test_0.slog")
df_test = pd.DataFrame(test)
df_test

,block_num,trial_num,stim_on_time,stim_on_error,resp,resp_time_time,resp_time_error,rt,correct,log_time,filename,in_out,loc,cond,type,pres_num,lag,log_num
0,0,0,6637.749435,0.0,J,6639.231155,0.000419,1.481720,True,6639.731155,out0371.jpg,outdoor,outdoor,once,lure,1,0,0
1,0,1,6639.775147,0.0,F,6640.488956,0.000445,0.713809,True,6640.988956,in0025.jpg,indoor,indoor,massed,target,1,1,0
2,0,2,6641.016720,0.0,J,6642.063590,0.001532,1.046870,True,6642.563590,in0397.jpg,indoor,indoor,spaced,lure,1,8,0
3,0,3,6642.591461,0.0,J,6643.505558,0.008421,0.914096,True,6644.005558,in0141.jpg,indoor,indoor,once,lure,1,0,0
4,0,4,6644.041690,0.0,F,6644.708279,0.000953,0.666589,True,6645.208279,out0075_new.jpg,outdoor,outdoor,massed,target,1,1,0
5,0,5,6645.246326,0.0,F,6645.897921,0.001220,0.651594,True,6646.397921,out1376.jpg,outdoor,outdoor,spaced,target,1,6,0
6,0,6,6646.427516,0.0,F,6647.173849,0.008435,0.746333,True,6647.673849,in0110.jpg,indoor,indoor,spaced,target,1,8,0
7,0,7,6647.701814,0.0,J,6648.890318,0.008964,1.188504,True,6649.390318,out0010.jpg,outdoor,outdoor,massed,lure,1,1,0
8,0,8,6649.411013,0.0,F,6650.113367,0.000497,0.702354,True,6650.613367,in0400.jpg,indoor,indoor,once,target,1,0,0
9,0,9,6650.645498,0.0,F,6651.261778,0.000440,0.616279,True,6651.761778,out0146_new.jpg,outdoor,outdoor,once,target,1,0,0
